In [1]:
import pandas as pd

df = pd.read_csv('predictions/imdb_pred.csv')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd

max_features=10000

# Splitting the data into train and test sets
X = df['text']
y = df['prediction']

# X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer
count_vectorizer = CountVectorizer(max_features=max_features)

# Fit and transform the text data
X_vector = count_vectorizer.fit_transform(X)
feature_names = count_vectorizer.get_feature_names_out()

In [27]:
from sklearn.tree import _tree
import numpy as np

def get_rules(tree, feature_names, class_names=None):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    def traverse_nodes(node, current_rule, rules):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            left_child = tree_.children_left[node]
            right_child = tree_.children_right[node]

            # Add condition for the left child
            left_rule = f"{name} <= {threshold:.3f}"
            traverse_nodes(left_child, current_rule + [left_rule], rules)

            # Add condition for the right child
            right_rule = f"{name} > {threshold:.3f}"
            traverse_nodes(right_child, current_rule + [right_rule], rules)
        else:
            # Leaf node
            rule = {
                'rule': ' & '.join(current_rule),
                'samples': int(tree_.n_node_samples[node])
            }
            if class_names:
                class_id = np.argmax(tree_.value[node])
                class_name = class_names[class_id]
                rule['class'] = class_name
            else:
                rule['response'] = tree_.value[node]
            rules.append(rule)

    rules = []
    traverse_nodes(0, [], rules)
    return rules



In [30]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(max_leaf_nodes=20, max_depth=10)
decision_tree.fit(X_vector, y)

rules = get_rules(decision_tree, feature_names, [0,1])
for r in rules:
    print(r)

{'rule': 'worst <= 0.500 & bad <= 0.500 & waste <= 0.500 & great <= 0.500 & awful <= 0.500 & boring <= 0.500 & terrible <= 0.500 & excellent <= 0.500 & poor <= 0.500 & lame <= 0.500', 'samples': 3930, 'class': 1}
{'rule': 'worst <= 0.500 & bad <= 0.500 & waste <= 0.500 & great <= 0.500 & awful <= 0.500 & boring <= 0.500 & terrible <= 0.500 & excellent <= 0.500 & poor <= 0.500 & lame > 0.500', 'samples': 64, 'class': 0}
{'rule': 'worst <= 0.500 & bad <= 0.500 & waste <= 0.500 & great <= 0.500 & awful <= 0.500 & boring <= 0.500 & terrible <= 0.500 & excellent <= 0.500 & poor > 0.500', 'samples': 197, 'class': 0}
{'rule': 'worst <= 0.500 & bad <= 0.500 & waste <= 0.500 & great <= 0.500 & awful <= 0.500 & boring <= 0.500 & terrible <= 0.500 & excellent > 0.500', 'samples': 325, 'class': 1}
{'rule': 'worst <= 0.500 & bad <= 0.500 & waste <= 0.500 & great <= 0.500 & awful <= 0.500 & boring <= 0.500 & terrible > 0.500', 'samples': 154, 'class': 0}
{'rule': 'worst <= 0.500 & bad <= 0.500 & was